In [ ]:
import pandas as pd
import numpy as np
import os
import random
import glob
import librosa
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import lightgbm as lgb
import xgboost as xgb
import optuna
from tqdm import tqdm

In [ ]:
def extract_feature(file_name):
    """Function Extracts Features from WAV file"""
    X, sample_rate = librosa.load(file_name, duration=2.97)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=128).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate, n_chroma=12, n_fft=2048).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate, n_mels=128, hop_length=512).T,axis=0)
    result=np.hstack((result, mel))
    return result

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

def gender(g):
    """Returns Gender Label"""
    if int(g[0:2]) % 2 == 0:
        return 'female'
    else:
        return 'male'

def load_data(test_size=0.2):
    """Loads Data from directory containing WAV files."""
    X_train, X_test, y_train, y_test=[], [], [], []
    x, y = [], []
    for file in tqdm(glob.glob("../data/ravdess/Actor_*/*.wav")):
        actor = file.split('/')[-2]
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]] #+ '_' + gender(file_name.split("-")[-1])
        feature=extract_feature(file)
        # use the same routine we used for squeezenet
        if actor == "Actor_11" or \
            actor == "Actor_12" or \
            actor == "Actor_23" or \
            actor == "Actor_24":
            X_train.append(feature)
            y_train.append(emotion)
        else:
            X_test.append(feature)
            y_test.append(emotion)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_data()

In [ ]:
X_train, X_test = np.array(X_train), np.array(X_test)
print((X_train.shape, X_test.shape))

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def print_confusion_matrix(
    model_abrv,
    confusion_matrix, 
    class_names, 
    figsize=(10, 7), 
    fontsize=14, 
    model='clf', 
    save=True,
):
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names
    )
    fig, ax = plt.subplots(1,1,figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, ax=ax, fmt='d')
    except ValueError:
        raise ValueError("Confusion matrix values must be integers")
    
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    b, t = plt.ylim()
    b += 0.5
    t -= 0.5
    plt.ylim(b, t)
    if save == True:
        plt.savefig('tuned_'+model_abrv[model]+'_confision_matrix.jpg')
    plt.show()

In [ ]:
def model(
    clf,
    model_abrv,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    models=models,
    save=False,
    print_stat=True,
    inc_train=False,
    cv=False,
):
    clf_model = models[clf]
    clf_model.fit(X_train, y_train)
    y_pred = clf_model.predict(X_test)
    
    if print_stat == True:
        print(model_abrv[clf])
        print('\nTest Stats\n', classification_report(y_test, y_pred))
        print_confusion_matrix(
            model_abrv,
            confusion_matrix(y_test, y_pred), 
            unique_labels(y_test, y_pred), 
            model=clf
        )
        if inc_train == True:
            print(model_abrv[clf])
            print('\nTrain Stats\n', classification_report(y_train, clf_model.predict(X_train)))
            print_confusion_matrix(
                model_abrv,
                confusion_matrix(y_train, clf_model.predict(X_train)), unique_labels(y_test, y_pred), 
                model=clf
            )
    if cv == True:
        print(
            model_abrv[clf] + ' CV Accuracy:',
            np.mean(cross_val_score(clf_model, X_train, y_train, cv=5, scoring='accuracy')),
        )
    if save == True:
        return clf_model

In [ ]:
lr_params = {'multi_class':'multinomial',
             'class_weight': None, 
             'solver': 'liblinear', 
             'max_iter':10000}

rg_params = {'solver': 'auto', 
             'max_iter':10000}

knn_params = {'weights':'distance',
             'n_neighbors': 32}

dt_params = {'criterion': 'entropy', 
             'max_depth': 35, 
             'min_samples_leaf': 4, 
             'min_samples_split': 23, 
             'max_leaf_nodes': 169}

lgb_params = {'num_leaves': 22, 
              'max_depth': 37, 
              'n_estimators': 12310, 
              'subsample_for_bin': 491645, 
              'min_data_in_leaf': 27, 
              'reg_alpha': 1.744123586157066, 
              'colsample_bytree': 0.6495503686746514, 
              'learning_rate': 0.8581745963346554, 
              'boosting_type': 'dart'}

xgb_params = {'booster': 'gbtree', 
              'lambda': 7.201651687969849e-08, 
              'alpha': 2.2495125443474775e-05, 
              'max_depth': 7, 
              'eta': 9.307925211476325e-06, 
              'gamma': 1.7948741419263195e-05, 
              'grow_policy': 'lossguide'}

rf_params = {'criterion': 'entropy', 
             'max_depth': 15, 
             'n_estimators': 22984, 
             'min_samples_leaf': 3, 
             'min_samples_split': 9, 
             'max_leaf_nodes': 239, 
             'random_state': 22}

svm_params = {'kernel': 'poly',
             'degree': 4,}



models = {'rg': RidgeClassifier(**rg_params),
          'dt':DecisionTreeClassifier(**dt_params), 
          'lgb':lgb.LGBMClassifier(**lgb_params), 
          'xgb':xgb.XGBClassifier(**xgb_params),
          'svm':SVC(**svm_params), 
          'kn':KNeighborsClassifier(**knn_params),  
          'lr':LogisticRegression(**lr_params),
          'rf':RandomForestClassifier(**rf_params),
}

model_abrv = {'rg': 'Ridge Classifier',
              'dt':'Decision Tree Classifier', 
              'lgb':'LGBM Classifier', 
              'xgb':'XGB Classifier', 
              'svm':'SVM Classifier',
              'kn':'K-Nearest Neighbors', 
              'lr':'Logistic Regression', 
              'rf':'Random Forest',
              'v' : 'ensemble of all models'
}
v_params = {'estimators':[
                        ('lr', models['lr']),
                        ('dt', models['dt']),
                        ('rf', models['rf']),
                        ('lgb', models['lgb']),
                        ('xgb', models['xgb']),
                        ('svm', models['svm']),
                        ('kn', models['kn']),
                        ('rg', models['rg']),
                ], 
             'voting':'soft'}
models['v'] = VotingClassifier(**v_params)

In [ ]:
for key in models.keys():
    model(key, model_abrv, cv=True, print_stat=False)

In [ ]:
for key in models.keys():
    model(key, model_abrv, inc_train=True)

# Hyperparameter Tuning

In [2]:
def objective_lgb(trial):
        params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 150),
        'max_depth': trial.suggest_int('max_depth', 2, 100),
        'n_estimators': trial.suggest_int('n_estimators', 10, 20000),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 100000, 500000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 100),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.0, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-0),
        'boosting_type': trial.suggest_categorical('boosting_type', ['goss','gbdt','dart']),
        'objective': 'multiclass',
        'verbose': -1,
        'random_state':22,
        }
   
    model = lgb.LGBMClassifier(**params, nthread = 4, n_jobs = -1) 
    
    model.set_params(**params)
    
    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgb, n_trials=12) 

In [ ]:
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
def objective_rf(trial):
    
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 25),
        'n_estimators': trial.suggest_int('n_estimators', 15000, 25000),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 50, 250),
        }
   
    model = RandomForestClassifier(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_rf, n_trials=5) 

In [ ]:
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))